In [2]:
import pandas as pd
import numpy as np
import csv
import json
import string
import random
import re
from scipy.stats import pearsonr
from matplotlib import pyplot

np.random.seed(1234)
random.seed(1234)

In [3]:
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import STOPWORDS
import string

porter = PorterStemmer()
punctuation_dictionary = {s:None for s in list(string.punctuation)}
punctuation_dictionary["-"] = "_"

punctuation_translator = str.maketrans(punctuation_dictionary)

def text_cleaner(text, punctuation_translator, stemmer):
    text = "".join(re.split('xa0',text))
    text = str(text).translate(punctuation_translator)
    text = text.lower()
    text = porter.stem(text)
    return(text)

In [9]:
SubSpeech_predicted = pd.read_csv('SubSpeech_predicted_update.csv')

In [319]:
#SubSpeech_predicted = pd.read_csv('SubSpeech_predicted_update_new.csv')

In [4]:
Speech = pd.read_csv('CampaignSpeechMerged_cleaned.csv')

In [147]:
Polls = pd.read_csv('Polls.csv')

In [434]:
AllCandidate = pd.read_csv('AllCandidates.csv')

In [9]:
#Speech.head()

In [10]:
#Polls.head()

In [372]:
corr, _ = pearsonr(SubSpeech_predicted['Pop_prob0'], SubSpeech_predicted['Pop_prob'])
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: 0.755


In [325]:
#SubSpeech_predicted[SubSpeech_predicted.Pop.isna()].sort_values('Dif').head(20)

In [399]:
SubSpeech_predicted.URL[821]

'/documents/remarks-rally-the-pensacola-bay-center-pensacola-florida'

In [398]:
SubSpeech_predicted0.clean_text[821]

'change is coming all the people whove rigged the system for their own personal benefit are trying to stop our change campaign because they know that their gravy train has reached its last stop its your turn now this is your time the fact that so many encrusted old political insiders oppose our campaign is the best proof you will ever need that we are fighting for real change – not partisan change we are fighting for all americans – democrats republicans independents conservatives liberals – whove been failed by this corrupt system were fighting for everyone who doesnt have a voice were also fighting for every region of this country for every part of florida and every part of america from pensacola to pittsburgh from baltimore to baton rouge we are fighting for every last city and every last person in this country hillary clinton is the candidate of the past ours is the campaign of the future in this future we are going to pursue new trade policies that put american workers first – and

## Speech Level

In [468]:
SubSpeech_predicted.Month.replace({'7.0': 'July', '8.0': 'August', '9.0': 'September', '10.0': 'October', '11.0': 'November', },
                        inplace=True)

In [149]:
Speech.Month.replace({'7.0': 'July', '8.0': 'August', '9.0': 'September', '10.0': 'October', '11.0': 'November', },
                        inplace=True)

In [469]:
SubSpeech_predicted['CandiYear'] = SubSpeech_predicted.Candidate + '_' + SubSpeech_predicted.Year.astype(str) 

In [328]:
CanditateYear = Polls.Candidate + '_' + Polls.Year.astype(str)

In [329]:
CandiPop_SubSpeech = SubSpeech_predicted[SubSpeech_predicted.CandiYear.isin(CanditateYear)]

In [330]:
len(CandiPop_SubSpeech.Speech_index.unique())

3494

In [331]:
CandiPop_SubSpeech = CandiPop_SubSpeech[['Candidate','Chanel','City','Year','Month','Day','Speech_index',
                                         'Subspeech_index', 'Title','Type','Pop_class']]
                         


In [144]:
CandiPop_SubSpeech.to_csv('Subspeech_Level_Pop_MajorCand.csv', index=False)

In [332]:
#CandiPop_SubSpeech.to_csv('Subspeech_Level_Pop_MajorCand_new.csv', index=False)

In [333]:
Pop_prop = []
SubSpeechCounts = []
for x in Speech.Speech_index:
    Pop_prop.append(np.mean(SubSpeech_predicted.Pop_class[SubSpeech_predicted.Speech_index == x]))
    SubSpeechCounts.append(len(SubSpeech_predicted.Subspeech_index[SubSpeech_predicted.Speech_index == x]))

In [334]:
Pop_Speech = Speech.assign(Pop_prop = Pop_prop)
Pop_Speech = Pop_Speech.assign(SubSpeechCounts = SubSpeechCounts)

In [10]:
#Pop_Speech.head()

### Add speech word counts

In [187]:
Speech['cleantext'] = Speech["Content"].apply(lambda x: text_cleaner(x, punctuation_translator, porter))

In [188]:
Speech['Speech_len'] = [len(x.split(' ')) for x in Speech['cleantext']]

In [189]:
Speech_len = Speech[['Speech_index','Speech_len']]

In [335]:
Pop_Speech = Pop_Speech.merge(Speech_len)

In [336]:
Pop_Speech = Pop_Speech[['Candidate','Chanel','City','Year','Month','Day','Speech_index',
                         'Title','Type','Pop_prop','SubSpeechCounts','Speech_len']]

In [337]:
Pop_Speech.Month.unique()

array(['November', 'October', 'September', 'August', 'July', 'June',
       'March', 'May', 'April', 'February', 'January', 'December', nan],
      dtype=object)

In [369]:
len(Pop_Speech[Pop_Speech.Year>=1952].Title)

4324

In [122]:
Pop_Speech.to_csv("Speech_Level_Pop_All.csv", index=False)

In [338]:
Pop_Speech.to_csv("Speech_Level_Pop_All_new.csv", index=False)

### Select Relevant Speeches

In [194]:
set(Polls.Candidate).difference(set(Pop_Speech.Candidate))

{'Alfred M. Landon',
 'Gary Johnson',
 'George Wallace',
 'John Anderson',
 'Ralph Nader',
 'Strom Thurmond'}

In [339]:
Pop_Speech['CandiYear'] = Pop_Speech.Candidate + '_' + Pop_Speech.Year.astype(str) 

In [340]:
CanditateYear = Polls.Candidate + '_' + Polls.Year.astype(str)

In [341]:
CandiPop_Speech = Pop_Speech[Pop_Speech.CandiYear.isin(CanditateYear)]

In [342]:
len(CandiPop_Speech.Title)

3494

In [343]:
CandiPop_Speech.Speech_len.describe()

count     3494.000000
mean      2167.556382
std       1369.384238
min         25.000000
25%       1142.250000
50%       1984.000000
75%       2905.750000
max      11057.000000
Name: Speech_len, dtype: float64

In [344]:
CandiPop_Speech.sort_values(by=['Pop_prop'], ascending=False).head(10)

,Candidate,Chanel,City,Year,Month,Day,Speech_index,Title,Type,Pop_prop,SubSpeechCounts,Speech_len,CandiYear
57,Donald J. Trump,NaN,NaN,2016,October,11.0,Speech57,\nExcerpts of Remarks at Aaron Bessant Park in...,Campaign Documents,1.0,3,1088,Donald J. Trump_2016
950,John McCain,NaN,NaN,2008,September,18.0,Speech950,"\nRemarks in Cedar Rapids, Iowa\n",Campaign Documents,1.0,1,982,John McCain_2008
3477,George Bush,STUMP,Findlay,1992,September,26.0,Speech4353,NaN,SPEECH,1.0,1,306,George Bush_1992
29,Donald J. Trump,NaN,NaN,2016,October,29.0,Speech29,\nRemarks at the Jeffco Fairgrounds Event Cent...,Campaign Documents,1.0,4,768,Donald J. Trump_2016
2538,George McGovern,STUMP,CLEVELAND,1972,October,25.0,Speech3414,NaN,SPEECH,1.0,1,250,George McGovern_1972
963,Barack Obama,NaN,NaN,2008,September,12.0,Speech963,"\nRemarks in Dover, New Hampshire\n",Campaign Documents,1.0,2,1834,Barack Obama_2008
962,John McCain,NaN,NaN,2008,September,13.0,Speech962,\nSenator McCain's Weekly Radio Address\n,Campaign Documents,1.0,1,742,John McCain_2008
901,Barack Obama,NaN,NaN,2008,October,11.0,Speech901,"\nRemarks in Philadelphia, Pennsylvania\n",Campaign Documents,1.0,2,1560,Barack Obama_2008
1651,John F. Kerry,NaN,NaN,2004,March,9.0,Speech1651,"\nRemarks Following the Florida, Louisiana, Mi...",Campaign Documents,1.0,1,652,John F. Kerry_2004
2318,Hubert H. Humphrey,STUMP,WATTS,1968,October,23.0,Speech3194,NaN,SPEECH,1.0,1,409,Hubert H. Humphrey_1968


In [75]:
CandiPop_Speech.to_csv("Speech_Level_Pop_MajorCand.csv", index = False)

In [345]:
#CandiPop_Speech.to_csv("Speech_Level_Pop_MajorCand_new.csv", index = False)

## Candidate level

In [346]:
CandiPop = SubSpeech_predicted.groupby(['Candidate','Year']).Pop_class.mean().reset_index()

In [347]:
CandiCounts = SubSpeech_predicted.groupby(['Candidate','Year']).Subspeech_index.count().reset_index()

In [348]:
CandiSpeechCounts = Speech.groupby(['Candidate','Year']).Speech_index.count().reset_index()

In [349]:
CandiPop = CandiPop.merge(CandiCounts).merge(CandiSpeechCounts)
CandiPop.head()

,Candidate,Year,Pop_class,Subspeech_index,Speech_index
0,Adlai Stevenson,1952,0.000000,409,186
1,Adlai Stevenson,1956,0.012097,248,75
2,Al Smith,1928,0.000000,9,1
3,"Albert Gore, Jr.",1999,0.000000,7,1
4,"Albert Gore, Jr.",2000,0.021277,188,14


In [350]:
len(CandiPop.Subspeech_index)

164

In [351]:
CandiPop.rename(columns={'Pop_class': 'Pop_prop', 'Subspeech_index': 'Subspeech_counts', 'Speech_index': 'Speech_counts' }, 
                      inplace = True)

In [94]:
CandiPop.to_csv('AllCandidateLevel.csv', index=False)

In [435]:
CandiPop = pd.read_csv('AllCandidateLevel.csv')

In [437]:
AllCandidate['CandiYear'] = AllCandidate.Candidate + '_' + AllCandidate.Year.astype(str) 

In [442]:
set(AllCandidate.Candidate).difference(set(CandiPop.Candidate))

{'Alfred M. Landon',
 'Gary Johnson',
 'George Wallace',
 'Jim Gilmore',
 'John Anderson',
 'Ralph Nader',
 'Strom Thurmond'}

In [445]:
#set(CandiPop.CandiYear).difference(set(AllCandidate.CandiYear))

In [446]:
CandiPop_All = CandiPop.merge(AllCandidate[['Party','Final','P2P','CandiYear']])

In [457]:
#CandiPop.sort_values('Pop_prop').tail(30)
CandiPop_All = CandiPop_All[CandiPop_All.Speech_counts > 4]

In [461]:
CandiPop_All.to_csv('AllCandidate_Year_pop.csv', index = False)

In [471]:
AllCandi_SubSpeech = SubSpeech_predicted[SubSpeech_predicted.CandiYear.isin(CandiPop_All.CandiYear)]
AllCandi_SubSpeech = AllCandi_SubSpeech[['Candidate','Chanel','City','Year','Month','Day','Speech_index',
                                         'Subspeech_index', 'Title','Type','Pop_class', 'CandiYear']]

In [475]:
AllCandi_SubSpeech = AllCandi_SubSpeech.merge(AllCandidate[['Party','Final','P2P','CandiYear']])

In [476]:
AllCandi_SubSpeech.to_csv('AllCandidate_SubSpeech_Pop.csv', index = False)

## Select final candidates

In [5]:
CandiPop = pd.read_csv('AllCandidateLevel.csv')

In [352]:
CanditateYear = Polls.Candidate + '_' + Polls.Year.astype(str)

In [83]:
#CandiPop[CandiPop.Speech_index >5].sort_values('Pop_class').tail(30)

In [353]:
CandiPop['CandiYear'] = CandiPop.Candidate + '_' + CandiPop.Year.astype(str) 

In [354]:
CandiPop_final = CandiPop[CandiPop.CandiYear.isin(CanditateYear)]

In [355]:
CandiPop_final.rename(columns={'Pop_class': 'Pop_prop', 'Subspeech_index': 'Subspeech_counts', 'Speech_index': 'Speech_counts' }, 
                      inplace = True)

//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [356]:
CandiPop_final.sort_values('Pop_prop', ascending= False).head(12)

,Candidate,Year,Pop_prop,Subspeech_counts,Speech_counts,CandiYear
34,Donald J. Trump,2016,0.313725,816,75,Donald J. Trump_2016
8,Barack Obama,2008,0.132184,870,185,Barack Obama_2008
48,George McGovern,1972,0.070312,256,98,George McGovern_1972
99,Michael S. Dukakis,1988,0.046154,260,66,Michael S. Dukakis_1988
88,John McCain,2008,0.043924,683,143,John McCain_2008
82,John F. Kerry,2004,0.036675,409,88,John F. Kerry_2004
150,Walter F. Mondale,1984,0.029046,241,67,Walter F. Mondale_1984
12,Barack Obama,2012,0.026902,1078,205,Barack Obama_2012
4,"Albert Gore, Jr.",2000,0.021277,188,14,"Albert Gore, Jr._2000"
68,Jimmy Carter,1976,0.019737,152,55,Jimmy Carter_1976


In [357]:
CandiPop_final[CandiPop_final.Year <=1996].sort_values('Pop_prop', ascending= False).head(12)

,Candidate,Year,Pop_prop,Subspeech_counts,Speech_counts,CandiYear
48,George McGovern,1972,0.070312,256,98,George McGovern_1972
99,Michael S. Dukakis,1988,0.046154,260,66,Michael S. Dukakis_1988
150,Walter F. Mondale,1984,0.029046,241,67,Walter F. Mondale_1984
68,Jimmy Carter,1976,0.019737,152,55,Jimmy Carter_1976
156,William J. Clinton,1992,0.012780,313,76,William J. Clinton_1992
45,George Bush,1992,0.012195,328,126,George Bush_1992
1,Adlai Stevenson,1956,0.012097,248,75,Adlai Stevenson_1956
126,Robert Dole,1996,0.008818,567,111,Robert Dole_1996
117,Richard Nixon,1968,0.005618,356,67,Richard Nixon_1968
63,Hubert H. Humphrey,1968,0.004032,744,154,Hubert H. Humphrey_1968


In [406]:
len(CandiPop_final.Candidate)

41

In [358]:
CandiPop_final.groupby(['Year']).Pop_prop.mean()

Year
1936    0.000000
1940    0.000000
1944    0.000000
1948    0.000000
1952    0.001222
1956    0.006048
1960    0.000000
1964    0.000000
1968    0.004825
1972    0.035156
1976    0.009868
1980    0.000000
1984    0.014523
1988    0.023077
1992    0.008325
1996    0.004409
2000    0.010638
2004    0.018337
2008    0.088054
2012    0.016363
2016    0.160613
Name: Pop_prop, dtype: float64

In [134]:
CandiPop_final.to_csv('CandidateMerged_Level.csv', index= False)

In [359]:
CandiPop_final.to_csv('CandidateMerged_Level_new.csv', index= False)

In [315]:
CandiPop_final = pd.read_csv('CandidateMerged_Level_new.csv') 

## Candidate Month

In [360]:
SubSpeech_predicted.Month.unique()

array(['November', 'October', 'September', 'August', 'July', 'June',
       'March', 'May', 'April', 'February', 'January', 'December', nan],
      dtype=object)

In [361]:
CandiPopMonth = SubSpeech_predicted.groupby(['Candidate','Year','Month']).Pop_class.mean().reset_index()

In [362]:
CandiCountsMonth = SubSpeech_predicted.groupby(['Candidate','Year', 'Month']).Subspeech_index.count().reset_index()

In [363]:
CandiSpeechCountsMonth = Speech.groupby(['Candidate','Year','Month']).Speech_index.count().reset_index()

In [364]:
CandiPopMonth = CandiPopMonth.merge(CandiCountsMonth).merge(CandiSpeechCountsMonth)
CandiPopMonth.head()

,Candidate,Year,Month,Pop_class,Subspeech_index,Speech_index
0,Adlai Stevenson,1952,August,0.0,10,4
1,Adlai Stevenson,1952,July,0.0,3,2
2,Adlai Stevenson,1952,November,0.0,10,4
3,Adlai Stevenson,1952,October,0.0,271,124
4,Adlai Stevenson,1952,September,0.0,109,46


In [365]:
CandiPopMonth.rename(columns={'Pop_class': 'Pop_prop', 'Subspeech_index': 'Subspeech_counts', 'Speech_index': 'Speech_counts' }, 
                      inplace = True)

In [366]:
CandiPopMonth['CandiYear'] = CandiPopMonth.Candidate + '_' + CandiPopMonth.Year.astype(str) 

In [367]:
CandiPopMonth = CandiPopMonth[CandiPopMonth.CandiYear.isin(CanditateYear)]

In [132]:
CandiPopMonth.to_csv('CandidateYearMonth_Level_Pop.csv', index=False)

In [368]:
CandiPopMonth.to_csv('CandidateYearMonth_Level_Pop_new.csv', index=False)

## Average Year Level 

In [179]:
PopYear_level = SubSpeech_predicted.groupby(['Year']).Pop_class.mean().reset_index()

In [181]:
PopYear_level.Pop_class.describe()

count    48.000000
mean      0.017818
std       0.042235
min       0.000000
25%       0.000000
50%       0.000000
75%       0.012120
max       0.184961
Name: Pop_class, dtype: float64

In [182]:
PopYear_level.to_csv('PopYear_level.csv', index =False)

In [11]:
len(SubSpeech_predicted[SubSpeech_predicted.Candidate == "Donald J. Trump"].Speech_index.unique())

87